# Clustering with GPT-3.5-turbo, GPT-4, and Langchain

In [1]:
import os
import tiktoken
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from dotenv import load_dotenv
from openai.embeddings_utils import cosine_similarity
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### Dataset is 1,200 DS related job posting from Indeed

In [3]:
# df = pd.read_csv('./data/data_science_jobs_indeed_usa.csv')

In [4]:
# encoding = tiktoken.get_encoding('cl100k_base')

In [5]:
# df = df.assign(token_count=df['Description'].apply(lambda x: len(encoding.encode(x))))

In [6]:
# print(df.shape)
# df.head()

(1200, 11)


,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,NaN,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25
2,2,IT Business Intelligence Developer (FT) Remote...,Ballad Health,"Remote in Blountville, TN",3.0,PostedPosted 30+ days ago,NaN,Job Details Apply Save Print this job Email a…,https://www.indeed.com/rc/clk?jk=58612836c63b8...,Job Details\nApply\nSave\nPrint this job\nEmai...,10
3,3,Data Engineer,Longevity Holdings Inc.,"Remote in Minneapolis-Saint Paul, MN",NaN,PostedPosted 3 days ago,"$90,000 - $110,000 a year",Incorporate core data management competencies ...,https://www.indeed.com/company/TwentyFirst/job...,Position: Data Engineer\nLocation: MN\nAs a Da...,29
4,4,Network Administrator/dba developer,WKI Kenworth,"Wichita, KS 67219",NaN,EmployerActive 2 days ago,"$50,000 - $70,000 a year",The Network Administrator provides 2nd level e...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Full Job Description\nThe Network Administrato...,28


In [3]:
# Azure: 
# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
# def get_embedding(text) -> list[float]:
#     text = text.replace("\n", " ")
#     return openai.Embedding.create(input=text, engine=OpenAiEmbedding)["data"][0]["embedding"]

# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
# def get_embedding(text, model="text-embedding-ada-002"):
#    text = text.replace("\n", " ")
#    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [8]:
# df = df.assign(embedding=df['Description'].apply(lambda x: get_embedding(x, model='text-embedding-ada-002')))

In [9]:
# df.to_parquet("job_descriptions_with_embeddings.parquet")

In [3]:
df = pd.read_parquet("job_descriptions_with_embeddings.parquet")

In [4]:
df.shape

(1200, 12)

In [5]:
df.head()

,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count,embedding
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ..."
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25,"[-0.012073525227606297, -0.026480479165911674,..."
2,2,IT Business Intelligence Developer (FT) Remote...,Ballad Health,"Remote in Blountville, TN",3.0,PostedPosted 30+ days ago,None,Job Details Apply Save Print this job Email a…,https://www.indeed.com/rc/clk?jk=58612836c63b8...,Job Details\nApply\nSave\nPrint this job\nEmai...,10,"[-0.021908748894929886, -0.002960818586871028,..."
3,3,Data Engineer,Longevity Holdings Inc.,"Remote in Minneapolis-Saint Paul, MN",NaN,PostedPosted 3 days ago,"$90,000 - $110,000 a year",Incorporate core data management competencies ...,https://www.indeed.com/company/TwentyFirst/job...,Position: Data Engineer\nLocation: MN\nAs a Da...,29,"[-0.017482835799455643, -0.01076465379446745, ..."
4,4,Network Administrator/dba developer,WKI Kenworth,"Wichita, KS 67219",NaN,EmployerActive 2 days ago,"$50,000 - $70,000 a year",The Network Administrator provides 2nd level e...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Full Job Description\nThe Network Administrato...,28,"[0.0011034636991098523, -0.000915585900656879,..."


In [6]:
X = np.vstack(df.embedding.values)

* `n_clusters=6` is arbitrary & just for example

In [7]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=42)
df['kmeans_label'] = kmeans.fit_predict(X)

In [8]:
print(df.shape)
df.head(2)

(1200, 13)


,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count,embedding,kmeans_label
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ...",4
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25,"[-0.012073525227606297, -0.026480479165911674,...",2


In [33]:
# res_df.to_parquet("ClusterLabelerTestResults.parquet")

In [2]:
# import pandas as pd

# res_df = pd.read_parquet("ClusterLabelerTestResults.parquet")

In [2]:
# import langchain
# from langchain.cache import SQLiteCache
# langchain.llm_cache = SQLiteCache(database_path=".langchain.db")

In [14]:
import time
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [15]:
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


system_template = "You're an expert journalist. You're helping me write a concise topic title for job descriptions."
human_template = "Using the following job description, write a concise tag line.\n\nDESCRIPTIONS:{Description}\n\nTAG LINE:"

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(system_template),
        HumanMessagePromptTemplate.from_template(human_template),
    ],
    input_variables=["Description"],
)
    
chain = LLMChain(
    llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"), prompt=prompt, verbose=False
)


In [16]:
list_to_chain = df[["Description"]].to_dict('records')
len(list_to_chain)

1200

In [17]:
list_to_chain[0]

{'Description': 'You’ll be working alongside a team of eight analysts & data scientists - collaborating to design datasets, develop analytical models & provide insights.'}

In [18]:
sem = asyncio.Semaphore(50)

async def delay_wrapper(func, *args, **kwargs):
    async with sem:
        result = await func(*args, **kwargs)
    await asyncio.sleep(2)  # 2 second delay
    return result


In [92]:
from src_index.token_catcher import Usage
usage = Usage()

In [20]:
batch_size = 100
result = []

start_time = time.time()

for i, batch_start in enumerate(range(0, len(list_to_chain), batch_size)):
    batch_start_time = time.time()
    batch = list_to_chain[batch_start: batch_start + batch_size]
    processed_batch = await delay_wrapper(chain.aapply, batch)
    result.append(processed_batch)
    elapsed_time = time.time() - batch_start_time
    print(f"Processed batch {i + 1} of {len(list_to_chain) // batch_size}. Elapsed time: {elapsed_time:.2f} seconds")
    
final_time = time.time() - start_time
print(f"Total elapsed time: {final_time:.2f} seconds")
print(f"Tokens used: {usage.total_tokens():,}")

Processed batch 1 of 12. Elapsed time: 4.15 seconds


Retrying langchain.chat_models.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out.
Retrying langchain.chat_models.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out.


Processed batch 2 of 12. Elapsed time: 607.80 seconds
Processed batch 3 of 12. Elapsed time: 3.91 seconds
Processed batch 4 of 12. Elapsed time: 4.56 seconds
Processed batch 5 of 12. Elapsed time: 4.01 seconds
Processed batch 6 of 12. Elapsed time: 3.28 seconds
Processed batch 7 of 12. Elapsed time: 3.64 seconds
Processed batch 8 of 12. Elapsed time: 3.88 seconds
Processed batch 9 of 12. Elapsed time: 3.95 seconds
Processed batch 10 of 12. Elapsed time: 3.42 seconds
Processed batch 11 of 12. Elapsed time: 4.17 seconds
Processed batch 12 of 12. Elapsed time: 3.24 seconds
Total elapsed time: 650.01 seconds
Tokens used: 0


In [21]:
result[0]

[{'text': '"Join our dynamic team of analysts and data scientists to drive data-driven insights and innovation."'},
 {'text': '"Experienced Cloud Migration Specialist for SAAS Implementations"'},
 {'text': '"Exciting Job Opportunity: Apply Now for a Rewarding Career!"'},
 {'text': '"Data Management Expert: Ensuring Governance, Security, and Quality"'},
 {'text': '"Experienced Network Administrator: Expert in SQL, Linux, Debian, and Arduino"'},
 {'text': '"Data Science Expert: Driving Business Outcomes and Patient Impact"'},
 {'text': 'Experienced Data Engineer: Expertise in Report Development, Data Science, and Business Analytics'},
 {'text': '"Experienced Developer: Data Conversions and Consultative Support"'},
 {'text': '"Leading the Way in Data Science: Hiring, Training, and Retaining Top Talent"'},
 {'text': '"Experienced Data Science Leader: Building and Leading High-Performing Teams"'},
 {'text': '"Data Analysis Expert with Proven Experience in Designing and Implementing End-to-E

In [22]:
# Flatten the list
results = [item for sublist in result for item in sublist]

# Check if 'result' length is same as the original data
assert len(results) == len(df), "Length of the result does not match with the original data"


In [23]:
results[0]

{'text': '"Join our dynamic team of analysts and data scientists to drive data-driven insights and innovation."'}

In [24]:
df['llm_title'] = pd.DataFrame(results)

In [25]:
# df.to_parquet("cluster_test_llm_titles.parquet")

In [9]:
import pandas as pd

df = pd.read_parquet("cluster_test_llm_titles.parquet")

In [10]:
df.head(1)

,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count,embedding,kmeans_label,llm_title
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ...",4,"""Join our dynamic team of analysts and data sc..."


In [34]:
from tqdm import tqdm
batch_size = 1
result_slow = []

start_time = time.time()

for i, batch_start in tqdm(enumerate(range(0, len(list_to_chain), batch_size)), total=len(list_to_chain)//batch_size):
    try:
        batch = list_to_chain[batch_start: batch_start + batch_size]
        processed_batch = chain.run(batch)
        result_slow.append(processed_batch)
    # replace Exception with list of actual errors
    except Exception as e:
        result_slow.append({'text': str(e)})
        
   
final_time = time.time() - start_time
print(f"Total elapsed time: {final_time:.2f} seconds")

100%|██████████| 1200/1200 [34:54<00:00,  1.75s/it]   

Total elapsed time: 2094.38 seconds


In [5]:
df.head(2)

,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count,embedding,kmeans_label,llm_title
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ...",4,"""Join our dynamic team of analysts and data sc..."
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25,"[-0.012073525227606297, -0.026480479165911674,...",2,"""Experienced Cloud Migration Specialist for SA..."


In [93]:
map_template_string = """The following is a list of job descriptions.
{descriptions}

Based on this list of descriptions, please do 2 things: 
(1) identify groups of similar jobs
(2) give a concise title for each group

Helpful Answer:"""

reduce_template_string = template = """The following is a list of job titles:
{titles}

Take these and write a concise job title that summarizes them. Respond with a single string using 5 words or less. 

TITLE:"""
MAP_PROMPT = PromptTemplate(input_variables=["descriptions"], template=map_template_string)
REDUCE_PROMPT = PromptTemplate(input_variables=["titles"], template=reduce_template_string)

In [94]:
def run_mr(input_doc, MAP_PROMPT, REDUCE_PROMPT):
    
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    map_llm_chain = LLMChain(llm=llm, prompt=MAP_PROMPT)

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    reduce_llm_chain = LLMChain(llm=llm, prompt=REDUCE_PROMPT)

    # Takes a list of documents and combines them into a single string
    combine_documents_chain = StuffDocumentsChain(
            llm_chain=reduce_llm_chain,
            document_variable_name="titles")
    
    # Combines and iteravely reduces the mapped documents 
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `combine_documents_chain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=4000)

    # Combining documents by mapping a chain over them, then combining results
    combine_documents = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_llm_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="descriptions",
        # Return the results of the map steps in the output
        ### Bug: this currently does not work ###
        return_intermediate_steps=False)
        
    # Define Map=Reduce
    map_reduce = MapReduceChain(
        # Chain to combine documents
        combine_documents_chain=combine_documents,
        # Splitter to use for initial split
        text_splitter=text_splitter)
    
    return map_reduce.run(input_text=input_doc)

In [95]:
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

usage = Usage()

for c in df.kmeans_label.unique():

    descriptions_str = "\n\n".join(
        [
            f"{article['Description']}\n"
            for article in df.query(f"kmeans_label == {c}").to_dict(orient="records")
        ]
    )
    
    result = run_mr(descriptions_str, MAP_PROMPT, REDUCE_PROMPT)
    
    df.loc[df.kmeans_label == c, "topic_title_"] = result
    
print(f"Tokens Used: {usage.total_tokens():,}")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).
Tokens Used: 43,018


In [86]:
df.topic_title.value_counts()

topic_title
Data Science and Analytics Roles                                  555
Business Analysis and Intelligence Professionals                  211
Machine Learning and AI Job Roles                                 180
Database Administrator and Performance Optimization Specialist    146
IT and Data Professionals                                         108
Name: count, dtype: int64

In [96]:
df.topic_title_.value_counts()

topic_title_
Data Science and Strategy Manager              281
Data Science and Analytics Specialist          274
Business Analyst and Intelligence Developer    211
Machine Learning Expert                        180
Database Specialist                            146
Cloud Migration Specialist                     108
Name: count, dtype: int64

In [88]:
df.kmeans_label.value_counts()

kmeans_label
4    281
3    274
5    211
1    180
0    146
2    108
Name: count, dtype: int64

In [97]:
df.head()

,Unnamed: 0,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions,token_count,embedding,kmeans_label,llm_title,topic_title,topic_title_
0,0,Data Scientist,Driven Brands,"Benicia, CA",2.4,PostedPosted 26 days ago,None,You’ll be working alongside a team of eight an...,https://www.indeed.com/rc/clk?jk=74d176d595225...,We invite you to join us at Driven Brands!\nHe...,26,"[-0.0194996390491724, -0.0041993726044893265, ...",4,"""Join our dynamic team of analysts and data sc...",Data Science and Analytics Roles,Data Science and Strategy Manager
1,1,Business Analyst,Sabot Consulting,Remote,NaN,PostedPosted 4 days ago,$80 - $120 an hour,Preferred candidates will have prior experienc...,https://www.indeed.com/rc/clk?jk=f662b2efb509b...,Sabot Consulting (Sabot) is a management consu...,25,"[-0.012073525227606297, -0.026480479165911674,...",2,"""Experienced Cloud Migration Specialist for SA...",IT and Data Professionals,Cloud Migration Specialist
2,2,IT Business Intelligence Developer (FT) Remote...,Ballad Health,"Remote in Blountville, TN",3.0,PostedPosted 30+ days ago,None,Job Details Apply Save Print this job Email a…,https://www.indeed.com/rc/clk?jk=58612836c63b8...,Job Details\nApply\nSave\nPrint this job\nEmai...,10,"[-0.021908748894929886, -0.002960818586871028,...",2,"""Exciting Job Opportunity: Apply Now for a Rew...",IT and Data Professionals,Cloud Migration Specialist
3,3,Data Engineer,Longevity Holdings Inc.,"Remote in Minneapolis-Saint Paul, MN",NaN,PostedPosted 3 days ago,"$90,000 - $110,000 a year",Incorporate core data management competencies ...,https://www.indeed.com/company/TwentyFirst/job...,Position: Data Engineer\nLocation: MN\nAs a Da...,29,"[-0.017482835799455643, -0.01076465379446745, ...",4,"""Data Management Expert: Ensuring Governance, ...",Data Science and Analytics Roles,Data Science and Strategy Manager
4,4,Network Administrator/dba developer,WKI Kenworth,"Wichita, KS 67219",NaN,EmployerActive 2 days ago,"$50,000 - $70,000 a year",The Network Administrator provides 2nd level e...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Full Job Description\nThe Network Administrato...,28,"[0.0011034636991098523, -0.000915585900656879,...",0,"""Experienced Network Administrator: Expert in ...",Database Administrator and Performance Optimiz...,Database Specialist


In [98]:
from llama_index.schema import TextNode, Document

In [99]:
nodes = []

for index, row in df.iterrows():
    node = TextNode(text=row['Descriptions'],
                            id_=f"anchordata_idx_{index}",
                            metadata={"Summary": row['llm_title'],
                                      "Title": row['Title'],
                                      "Company": row['Company'],
                                      "Location": row['Location'],
                                      "Group Name": row['topic_title']})
    nodes.append(node)
    

In [100]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
)
from IPython.display import Markdown, display

In [101]:
from llama_index.llms import OpenAI
from llama_index.embeddings import LangchainEmbedding

llm=OpenAI(model="gpt-3.5-turbo", temperature=0)
embed_model = LangchainEmbedding(
    OpenAIEmbeddings()
)

service_context = ServiceContext.from_defaults(
    embed_model=embed_model,
    llm=llm,
    chunk_size=1024
)

In [102]:
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [103]:
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

In [104]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import (
    BaseRetriever,
    VectorIndexRetriever,
    KeywordTableSimpleRetriever,
)

from typing import List

In [105]:
class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both semantic search and hybrid search."""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        keyword_retriever: KeywordTableSimpleRetriever,
        mode: str = "AND",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._keyword_retriever = keyword_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        keyword_nodes = self._keyword_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        keyword_ids = {n.node.node_id for n in keyword_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in keyword_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(keyword_ids)
        else:
            retrieve_ids = vector_ids.union(keyword_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [106]:
from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine

# define custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=10)
keyword_retriever = KeywordTableSimpleRetriever(index=keyword_index)
custom_retriever = CustomRetriever(vector_retriever, keyword_retriever)

# define response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

# vector query engine
vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)
# keyword query engine
keyword_query_engine = RetrieverQueryEngine(
    retriever=keyword_retriever,
    response_synthesizer=response_synthesizer,
)

In [107]:
response = custom_query_engine.query("What are some jobs in management that are Remote?")

INFO:llama_index.indices.keyword_table.retrievers:> Starting query: What are some jobs in management that are Remote?
> Starting query: What are some jobs in management that are Remote?
> Starting query: What are some jobs in management that are Remote?
INFO:llama_index.indices.keyword_table.retrievers:query keywords: ['management', 'jobs', 'remote']
query keywords: ['management', 'jobs', 'remote']
query keywords: ['management', 'jobs', 'remote']
INFO:llama_index.indices.keyword_table.retrievers:> Extracted keywords: ['management', 'remote']
> Extracted keywords: ['management', 'remote']
> Extracted keywords: ['management', 'remote']


In [108]:
print(response)


1. Remote Project Manager
2. Remote Product Manager
3. Remote Operations Manager
4. Remote Business Development Manager
5. Remote Human Resources Manager
6. Remote Quality Assurance Manager
7. Remote Financial Manager
8. Remote Marketing Manager
9. Remote Sales Manager
10. Remote IT Manager


In [36]:
import nest_asyncio
nest_asyncio.apply()

In [37]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index import ServiceContext

In [109]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type

llm=OpenAI(model="gpt-3.5-turbo", temperature=0)
embed_model = LangchainEmbedding(
    OpenAIEmbeddings()
)

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])
service_context = ServiceContext.from_defaults(
    embed_model=embed_model,
    llm=llm,
    chunk_size=1024,
    callback_manager=callback_manager)

In [110]:
# build index and query engine
vector_query_engine = VectorStoreIndex(
    nodes, use_async=False, service_context=service_context
).as_query_engine(similarity_top_k=10)

**********
Trace: index_construction
    |_embedding ->  0.197334 seconds
    |_embedding ->  0.2072 seconds
    |_embedding ->  0.412453 seconds
    |_embedding ->  0.267314 seconds
    |_embedding ->  0.294713 seconds
    |_embedding ->  0.298592 seconds
    |_embedding ->  0.340853 seconds
    |_embedding ->  0.527856 seconds
    |_embedding ->  0.347137 seconds
    |_embedding ->  0.227518 seconds
    |_embedding ->  0.325264 seconds
    |_embedding ->  0.219555 seconds
    |_embedding ->  0.220645 seconds
    |_embedding ->  0.373602 seconds
    |_embedding ->  0.44774 seconds
    |_embedding ->  0.24762 seconds
    |_embedding ->  0.384562 seconds
    |_embedding ->  0.228622 seconds
    |_embedding ->  0.159505 seconds
    |_embedding ->  0.232922 seconds
    |_embedding ->  0.246764 seconds
    |_embedding ->  0.204765 seconds
    |_embedding ->  0.210807 seconds
    |_embedding ->  0.205868 seconds
    |_embedding ->  0.245324 seconds
    |_embedding ->  0.249035 seconds
    |

In [111]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="job_descriptions", description="Useful for answering questions about jobs"
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    use_async=True,
)

In [112]:
response = query_engine.query(
    "Review 10 data science jobs and summarize their similarities and differences?"
)

Generated 5 sub questions.
[job_descriptions] Q: What are the key responsibilities of a data science job?
[job_descriptions] Q: What are the required qualifications for a data science job?
[job_descriptions] Q: What are the common skills needed for a data science job?
[job_descriptions] Q: What are the typical salary ranges for data science jobs?
[job_descriptions] Q: What are the common job titles for data science roles?
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=37 request_id=c0b3276ae43eec602c65a4a5aeccbdee response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=37 request_id=c0b3276ae43eec602c65a4a5aeccbdee response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=37 request_id=c0b3276ae43eec602c65a4a5aeccbdee response_code=200
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=71 request_

In [113]:
from IPython.display import Markdown

In [114]:
Markdown(f"{response}")

Based on the context information provided, here is a summary of the similarities and differences among the 10 data science jobs:

Similarities:
- All 10 jobs require strong skills in programming, statistics, machine learning, and data wrangling.
- They all emphasize the importance of good communication skills and the ability to present findings to both technical and non-technical audiences.
- A degree in Computer Science, Engineering, or a relevant field is preferred for all positions.
- The jobs require experience in data analysis and working with large datasets.
- They all mention the need for problem-solving aptitude and an analytical mind.

Differences:
- The salary ranges vary significantly across the different jobs, with some offering hourly rates and others offering annual salaries.
- Some jobs require specific experience in fraud analytics, while others mention experience in areas like operations research, data mining, and trading/investing.
- The required qualifications for each job differ, with some jobs requiring US citizenship and an active Secret clearance, while others do not mention these requirements.
- The job titles vary, with positions like Data Scientist, Manager Data Science, and Azure Data Engineer being mentioned.

Overall, while there are similarities in the skills and qualifications required for these data science jobs, there are also notable differences in terms of specific experience, qualifications, and salary ranges.

In [115]:
example_job = df.Description.sample(1, random_state=8).tolist()
print(example_job[0])

Assist with the tuning and performance of the database environment, determining optimum values for database attributes.


In [116]:
comparison_prompt = """The following is job description we are interested in.

"Assist with the tuning and performance of the database environment, determining optimum values for database attributes."

Based on this job description, please do 2 things: 
(1) find similar job descriptions
(2) compare and contrast the similar jobs with the description from our job of interest

Helpful Comparison:"""

In [117]:
response = query_engine.query(
    comparison_prompt
)

Generated 2 sub questions.
[job_descriptions] Q: Find similar job descriptions
[job_descriptions] Q: Compare and contrast similar jobs with the description from our job of interest
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=26 request_id=7c8c1fa732247b32774ac9a4b583ebcf response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=26 request_id=7c8c1fa732247b32774ac9a4b583ebcf response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=26 request_id=7c8c1fa732247b32774ac9a4b583ebcf response_code=200
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=21 request_id=59b16a906595391808a6b813978c0683 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=21 request_id=59b16a906595391808a6b813978c0683 response_code=200
message='OpenAI API response' path=https

In [118]:
Markdown(f"{response}")

(1) Similar job descriptions:
1. Database Administrator at ABC Company: This role involves managing and maintaining the performance, integrity, and security of the company's databases. It includes tasks such as database tuning, optimizing queries, and determining optimal database configurations.

2. Database Performance Engineer at XYZ Corporation: This role focuses specifically on optimizing database performance. It involves analyzing database performance metrics, identifying bottlenecks, and implementing performance tuning strategies to improve database efficiency.

(2) Comparison and contrast:
The job description provided is similar to both the Database Administrator and Database Performance Engineer roles in terms of focusing on database tuning and performance optimization. However, there are some differences between the roles:

- The job of interest seems to have a broader scope, as it includes assisting with tuning and performance of the entire database environment, whereas the Database Administrator role may involve additional responsibilities such as database security and integrity maintenance.

- The Database Performance Engineer role, on the other hand, is more specialized and focused solely on optimizing database performance. This role may involve in-depth analysis of performance metrics and implementing specific strategies to improve efficiency.

Overall, while all three roles involve database tuning and performance optimization, the job of interest appears to have a broader scope, encompassing various aspects of the database environment, while the other two roles may have more specific focuses within the realm of database performance.

In [21]:
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.cache import SQLiteCache
from langchain.callbacks import get_openai_callback
langchain.llm_cache = SQLiteCache(database_path=".langchain.db")

In [22]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

#### Topics by Cluster Label

In [15]:
# sort clusters by their size, and create a new dataframe with the size and cluster number in sorted order. Maintain other columns
size_and_cluster_number = pd.DataFrame(df.groupby("kmeans_label").size().sort_values(ascending=False))

# print out 10 messages from n cluster
for cluster_number, size in size_and_cluster_number.iterrows():
    print(f"{cluster_number} has {size} messages")
    print(df[df.kmeans_label == cluster_number].sample(4).Description.values)
    print("\n")

4 has 0    281
Name: 4, dtype: int64 messages
['Familiarity with data visualization tools.\nComfortable pulling and manipulating data using SQL.\nParticipates in and oversees the Data Science team in analyzing…'
 "This team's key partners include other product teams, finance, data science, and other teams within data engineering."
 'A storyteller, be able to make sense of data and extrapolate meaningful and actionable insights.\nUse website data to create compelling stories and succinct…'
 'Set data standards and oversee stewardship, ensuring that all products and services have data standards built in from the start.']


3 has 0    274
Name: 3, dtype: int64 messages
['And methodology, and data visualization (PowerBI, Tableau, etc.).\nSalary: * depending on relevant industry experience.\nA Day In The Life of a Data Engineer I*.'
 'A Master’s degree or higher in a relevant field such as computer science, data science, applied statistics, or a quantitative social science field or a bachel

In [63]:
df.groupby('kmeans_label').agg({
    'token_count': ['sum', 'mean']
})

token_count           
                     sum       mean
kmeans_label                       
0                   3641  24.938356
1                   4766  26.477778
2                   3109  28.787037
3                   7529  27.478102
4                   7419  26.402135
5                   5266  24.957346

### Sample questions
We can sample around 4k tokens worth of questions. To do this, we shuffle the questions in each cluster, then cut the first ~4k token chunk from each cluster. This should give us ~50% sample ratio on average

In [17]:
from langchain.text_splitter import CharacterTextSplitter

def create_doc(messages, max_tokens=3800):
    #sample around 125 messages from the cluster
    input_doc = '\n\n'.join(messages)

    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=max_tokens, chunk_overlap=0, separator="\n\n")
    # Sanity check
    split_texts = text_splitter.split_text(input_doc)
    
    return split_texts[0]


# for clusters, create docs and save in list
docs = []
for cluster_number, size in size_and_cluster_number.iterrows():
    print(f"{cluster_number} has {size} messages")
    Description = df[df.kmeans_label == cluster_number].Description.values
    doc = create_doc(Description)
    docs.append(doc)

4 has 0    281
Name: 4, dtype: int64 messages
3 has 0    274
Name: 3, dtype: int64 messages
5 has 0    211
Name: 5, dtype: int64 messages
1 has 0    180
Name: 1, dtype: int64 messages
0 has 0    146
Name: 0, dtype: int64 messages
2 has 0    108
Name: 2, dtype: int64 messages


### Set up Map Reduce

In [18]:
from langchain import PromptTemplate


map_template_string = """The following is a list of texts that have been clustered using K-Means.
{questions}

Based on this list of questions, please do 3 things: 
(1) Identify the main theme
(2) Give a list of 3 to 5 main sub-themes
(3) Give a representitive example text in each sub-theme
(4) Estimate the proportion of questions that fall into each theme
(5) Share observations on outliers, i.e. questions that do not seem to belong

Helpful Answer:"""

MAP_PROMPT = PromptTemplate(input_variables=["questions"], template=map_template_string)

In [19]:
def run_map(input_doc):

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    map_llm_chain = LLMChain(llm=llm, prompt=MAP_PROMPT)
   
    return map_llm_chain.run(questions=input_doc)

* Single cluster test with GPT-3.5-turbo

In [23]:
from src_index.token_catcher import Usage
usage = Usage()

In [26]:
with get_openai_callback() as cb:
    gpt_35_test_result = run_map(docs[0])
    print(cb)
    print(gpt_35_test_result)
    

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
Main Theme: Data Science and Analytics

Sub-themes:
1. Data Management and Governance
   - Example text: Incorporate core data management competencies including data governance, data security, and data quality.
   - Proportion: 10%

2. Collaboration and Teamwork
   - Example text: Collaborate with central team data science, insight and CRM teams to employ insight, segmentations and models to improve personalization and targeting of CRM.
   - Proportion: 15%

3. Data Analysis and Insights
   - Example text: Analyze product data to identify problems and ideate product features and process enhancements.
   - Proportion: 20%

4. Data Infrastructure and Systems
   - Example text: Build controls, processes, and systems to ensure SLAs are met to manage and scale data pipelines from internal and external data sources.
   - Proportion: 15%

5. Data Science Strategy and Leadership
   - Example te

In [25]:
usage.total_tokens()

3880

### GPT-4 Map Reduce

In [112]:
gpt4_results = []

for input_doc in docs:
    result=run_map(input_doc)
    gpt4_results.append(result)

In [52]:
usage.total_tokens()

23889

In [114]:
#print results
for i, result in enumerate(gpt4_results):

    #print size of cluster
    print(f"Cluster {i} has {size_and_cluster_number.iloc[i].values[0]} messages: \n")

    #print the summary
    print(result)

    #print separator
    print("\n===============================================================\n\n")

Cluster 0 has 281 messages: 

Main Theme: Data Science and Analytics

Sub-themes:
1. Data Management and Governance
   - Example text: Incorporate core data management competencies including data governance, data security, and data quality.
   - Proportion: 10%

2. Collaboration and Teamwork
   - Example text: Collaborate with central team data science, insight and CRM teams to employ insight, segmentations and models to improve personalization and targeting of CRM.
   - Proportion: 15%

3. Data Analysis and Insights
   - Example text: Analyze product data to identify problems and ideate product features and process enhancements.
   - Proportion: 20%

4. Data Infrastructure and Systems
   - Example text: Build controls, processes, and systems to ensure SLAs are met to manage and scale data pipelines from internal and external data sources.
   - Proportion: 15%

5. Data Science Strategy and Leadership
   - Example text: Define a clear data science and analytics roadmap.
   - Proportion:

In [27]:
usage = Usage()

### GPT-3.5-Turbo Map Reduce

In [28]:
gpt35_results = []
with get_openai_callback() as cb:
    for input_doc in docs:
        result=run_map(input_doc)
        gpt35_results.append(result)
        print(cb)
        print(result)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
Main Theme: Data Science and Analytics

Sub-themes:
1. Data Management and Governance
   - Example text: Incorporate core data management competencies including data governance, data security, and data quality.
   - Proportion: 10%

2. Collaboration and Teamwork
   - Example text: Collaborate with central team data science, insight and CRM teams to employ insight, segmentations and models to improve personalization and targeting of CRM.
   - Proportion: 15%

3. Data Analysis and Insights
   - Example text: Analyze product data to identify problems and ideate product features and process enhancements.
   - Proportion: 20%

4. Data Infrastructure and Systems
   - Example text: Build controls, processes, and systems to ensure SLAs are met to manage and scale data pipelines from internal and external data sources.
   - Proportion: 15%

5. Data Science Strategy and Leadership
   - Example te

In [29]:
usage.total_tokens()

18945

In [30]:
#print results
for i, result in enumerate(gpt35_results):

    #print size of cluster
    print(f"Cluster {i} has {size_and_cluster_number.iloc[i].values[0]} messages: \n")

    #print the summary
    print(result)

    #print separator
    print("\n===============================================================\n\n")

Cluster 0 has 281 messages: 

Main Theme: Data Science and Analytics

Sub-themes:
1. Data Management and Governance
   - Example text: Incorporate core data management competencies including data governance, data security, and data quality.
   - Proportion: 10%

2. Collaboration and Teamwork
   - Example text: Collaborate with central team data science, insight and CRM teams to employ insight, segmentations and models to improve personalization and targeting of CRM.
   - Proportion: 15%

3. Data Analysis and Insights
   - Example text: Analyze product data to identify problems and ideate product features and process enhancements.
   - Proportion: 20%

4. Data Infrastructure and Systems
   - Example text: Build controls, processes, and systems to ensure SLAs are met to manage and scale data pipelines from internal and external data sources.
   - Proportion: 15%

5. Data Science Strategy and Leadership
   - Example text: Define a clear data science and analytics roadmap.
   - Proportion:

### Send all cases through Map Reduce chain

In [31]:
from langchain.text_splitter import CharacterTextSplitter
input_doc = '\n\n'.join(docs)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=4000,chunk_overlap=0,separator="\n\n")
# Sanity check
len(text_splitter.split_text(input_doc))

6

In [32]:
map_template_string = """The following is a list of texts that have been clustered using K-Means
{questions}

Based on this list of texts, please do 4 things: 
(1) identify the main themes 
(2) give a represntitive example question in each theme
(3) estimate the proportion of questions that fall into each theme
(4) share observations on outliers, i.e. questions that do not seem to belong

Helpful Answer:"""

reduce_template_string = template = """The following is a list of summaries for questions entered into a Q+A system:
{question_summaries}

Take these and distill it into a final, consolidated list with: 
(1) the main question themes 
(2) two represntitive example questions in each theme
(3) estimate the proportion of questions that fall into each theme
(4) a summary of outliers

Helpful Answer:"""
MAP_PROMPT = PromptTemplate(input_variables=["questions"], template=map_template_string)
REDUCE_PROMPT = PromptTemplate(input_variables=["question_summaries"], template=reduce_template_string)

In [33]:
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import (
                StuffDocumentsChain,
                LLMChain,
                ReduceDocumentsChain,
                MapReduceDocumentsChain,
            )


def run_map_reduce(input_doc, model, MAP_PROMPT, REDUCE_PROMPT):
    
    llm = ChatOpenAI(model_name=model, temperature=0)
    map_llm_chain = LLMChain(llm=llm, prompt=MAP_PROMPT)

    llm = ChatOpenAI(model_name=model, temperature=0)
    reduce_llm_chain = LLMChain(llm=llm, prompt=REDUCE_PROMPT)

    # Takes a list of documents and combines them into a single string
    combine_documents_chain = StuffDocumentsChain(
            llm_chain=reduce_llm_chain,
            document_variable_name="question_summaries")
    
    # Combines and iteravely reduces the mapped documents 
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `combine_documents_chain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=4000)

    # Combining documents by mapping a chain over them, then combining results
    combine_documents = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_llm_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="questions",
        # Return the results of the map steps in the output
        ### Bug: this currently does not work ###
        return_intermediate_steps=False)
        
    # Define Map=Reduce
    map_reduce = MapReduceChain(
        # Chain to combine documents
        combine_documents_chain=combine_documents,
        # Splitter to use for initial split
        text_splitter=text_splitter)
    
    return map_reduce.run(input_text=input_doc)

In [34]:
len(input_doc)

117588

In [35]:
len(docs[0])

20346

In [36]:
usage = Usage()

In [37]:
map_reduce_gpt4_res = []

with get_openai_callback() as cb:
    for i in range(len(docs)):
        result = run_map_reduce(docs[i], "gpt-4",MAP_PROMPT, REDUCE_PROMPT)
        map_reduce_gpt4_res.append(result)
        print(cb)
        print(f"\nCluster: {i}:")
        print(result)
        print("\n===============================================================\n\n")

Tokens Used: 5059
	Prompt Tokens: 4146
	Completion Tokens: 913
Successful Requests: 2
Total Cost (USD): $0.17915999999999999

Cluster: 0:
(1) Main Question Themes:
   - Data Science Strategy and Management
   - Data Analysis and Insights
   - Data Engineering and Infrastructure
   - Collaboration and Teamwork
   - Data Governance and Security

(2) Representative Example Questions:
   - Data Science Strategy and Management: 
     1. "How do you define and execute a data science strategy that ensures our organization's success?"
     2. "What are the key elements in setting up a successful data science workflow?"
   - Data Analysis and Insights: 
     1. "How do you analyze product data to identify problems and ideate product features?"
     2. "What methods do you use to turn raw data into actionable insights?"
   - Data Engineering and Infrastructure: 
     1. "How do you manage and scale data pipelines from internal and external data sources?"
     2. "What are the best practices for 

In [38]:
usage.total_tokens()

29513

#### GPT-3.5-turbo

In [39]:
usage = Usage()

In [40]:
map_reduce_gpt35_res = []

with get_openai_callback() as cb:
    for i in range(len(docs)):
        result = run_map_reduce(docs[i], "gpt-3.5-turbo", MAP_PROMPT, REDUCE_PROMPT)
        map_reduce_gpt35_res.append(result)
        print(cb)
        print(f"\nCluster: {i}:")
        print(result)
        print("\n===============================================================\n\n")

Tokens Used: 4575
	Prompt Tokens: 3930
	Completion Tokens: 645
Successful Requests: 2
Total Cost (USD): $0.0071849999999999995

Cluster: 0:
(1) Main Question Themes:
- Data Science Strategy and Leadership
- Data Management and Quality
- Collaboration and Teamwork
- Data Analysis and Insights
- Data Infrastructure and Tools

(2) Representative Example Questions:
- Data Science Strategy and Leadership:
  - "How can we define and execute a data science and analytics roadmap?"
  - "What are the best practices for building a data-driven culture within an organization?"

- Data Management and Quality:
  - "How can we ensure data governance, data security, and data quality?"
  - "What are the key steps in data cleaning and preprocessing to improve data quality?"

- Collaboration and Teamwork:
  - "How can we collaborate with other teams to turn data into critical information and knowledge?"
  - "What are effective strategies for fostering cross-functional collaboration in data science project

In [41]:
usage.total_tokens()

27171

### GPT-3.5-turbo

In [43]:
from langchain.text_splitter import CharacterTextSplitter
input_doc = '\n\n'.join(docs)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=4000,chunk_overlap=0,separator="\n\n")
# Sanity check
len(text_splitter.split_text(input_doc))

6

### Targeted Topic Test

In [44]:
reduce_template_string = template = """The following is a list of summaries:
{question_summaries}

Take these and distill it into a final, consolidated list with: 
(1) the top 5 texts related to AI and ML applications.
(2) estimate the proportion of each question

Helpful Answer:"""
REDUCE_PROMPT = PromptTemplate(input_variables=["question_summaries"], template=reduce_template_string)

#### GPT-4

In [45]:
with get_openai_callback() as cb:
    result = run_map_reduce(docs[3], "gpt-4", MAP_PROMPT, REDUCE_PROMPT)
    print(cb)
    print(result)

Tokens Used: 717
	Prompt Tokens: 482
	Completion Tokens: 235
Successful Requests: 1
Total Cost (USD): $0.028559999999999995
(1) Top 5 Texts Related to AI and ML Applications:
   - "What experience do you have in developing and deploying machine learning models for ad recommendation?"
   - "Can you provide examples of your proficiency with data mining and machine learning?"
   - "What is your understanding of various advanced analytical and machine learning methods?"
   - "Do you have experience with machine learning frameworks such as PyTorch or TensorFlow/Keras?"
   - "Can you describe your industry or research experience in data analytics, machine learning, and software development?"

(2) Estimated Proportion of Each Question:
   - "What experience do you have in developing and deploying machine learning models for ad recommendation?" - 30%
   - "Can you provide examples of your proficiency with data mining and machine learning?" - 20%
   - "What is your understanding of various adva

### GPT3.5 Turbo

In [46]:
usage = Usage()

In [47]:
with get_openai_callback() as cb:
    result = run_map_reduce(docs[3], "gpt-3.5-turbo", MAP_PROMPT, REDUCE_PROMPT)
    print(cb)
    print(result)

Tokens Used: 610
	Prompt Tokens: 509
	Completion Tokens: 101
Successful Requests: 1
Total Cost (USD): $0.0009655000000000001
(1) The top 5 texts related to AI and ML applications are:
- Strategic Communications
- Ad Recommendation
- Natural Language Processing
- Computer Vision
- Image Processing

(2) The estimated proportion of each question is:
- Machine Learning Expertise: 30%
- Data Science and Analytics: 25%
- AI and ML Applications: 30%
- Machine Learning Infrastructure: 15%
- Outliers: 0% (since they do not fit into any specific theme)


In [48]:
usage.total_tokens()

610